In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, re
import gc
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.preprocessing import StandardScaler

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

In [3]:
path = 'C:/final_prj/open/'
files = os.listdir(path)
parquet_files = [f for f in files if re.search(r'\.parquet$', f)]


In [4]:
def null_check(df) :
    missing_strs = ['NA', 'NaN', 'null', 'N/A', '', ' ']
    df = df.replace(missing_strs, pd.NA)
    null_cols = df.columns[df.isnull().any()]
    for col in null_cols  :
        n_counts = df[col].isnull().sum()
        print(f'null counts : {n_counts :,}')
        print(f'null ratio : {n_counts/len(df) :.2%}')
        print(df[col].value_counts())
        print('########' * 8)

In [5]:
def encode(df, method) :
    if method == 'label' :
        # 명목형 데이터 : 라벨 인코딩 (간단한 경우)
        for col in obj_cols:
            print(x[col].value_counts())
            x[col] = LabelEncoder().fit_transform(df[col])
            print(x[col].value_counts())
    if method == 'ordinal' :
        # 순서형 데이터 : 각 값의 숫자로 인코딩
        for col in obj_cols:
            print(x[col].value_counts())
            x[col] = x[col].str.extract(r'(\d+)').astype(int)
            print(x[col].value_counts())
    return x

In [6]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

def vif(x) :
    # 전처리

    # x = x.loc[:, x.nunique() > 1]  # 상수 컬럼 제거
    # x = x.replace([np.inf, -np.inf], np.nan).dropna()  # 결측/무한값 제거
    
    # 상수항 추가
    x_const = add_constant(x)
    
    # VIF 계산
    vif = pd.DataFrame()
    vif["Feature"] = x_const.columns
    vif["VIF"] = [variance_inflation_factor(x_const.values, i) for i in range(x_const.shape[1])]
    print(vif.sort_values("VIF", ascending=False))
    return vif

## 성과정보

In [7]:
parquet_files

['test_마케팅정보_total.parquet',
 'train_마케팅정보_total.parquet',
 'train_성과정보_total.parquet',
 '회원정보_total.parquet']

In [10]:
df = pd.read_parquet(path + 'train_성과정보_total.parquet')
df.head()

,기준년월,ID,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용건수_카드론_전월,증감율_이용금액_신용_전월,...,변동률_RV일시불평잔,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M
0,201807,TRAIN_000000,0.454548,0.454548,0.364727,1.999996,0.0,0.0,0.0,0.409766,...,0.999998,1.042805,0.999700,0.999998,0.999998,0.261886,0.270752,0.000000,1.044401,1.280543
1,201807,TRAIN_000001,-0.511808,-0.511808,-0.511808,0.000000,0.0,0.0,0.0,-1.010817,...,1.092698,0.905663,0.999998,0.999998,0.999998,-0.563388,-0.670348,0.000000,0.000000,0.000000
2,201807,TRAIN_000002,-0.062716,-0.062716,0.067843,-1.999996,0.0,0.0,0.0,-0.902166,...,1.006124,1.993590,0.852567,0.999998,0.999998,-0.046516,0.058114,-0.014191,0.524159,1.208420
3,201807,TRAIN_000003,0.135707,0.135707,0.153975,-1.999996,0.0,0.0,0.0,-0.866052,...,0.999998,1.050646,0.999877,0.999998,0.999998,0.023821,0.258943,0.000000,0.880925,1.657124
4,201807,TRAIN_000004,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000028,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,NaN,NaN


In [11]:
df.shape

(2400000, 49)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2400000 entries, 0 to 399999
Data columns (total 49 columns):
 #   Column              Dtype  
---  ------              -----  
 0   기준년월                int64  
 1   ID                  object 
 2   증감율_이용건수_신용_전월      float64
 3   증감율_이용건수_신판_전월      float64
 4   증감율_이용건수_일시불_전월     float64
 5   증감율_이용건수_할부_전월      float64
 6   증감율_이용건수_CA_전월      float64
 7   증감율_이용건수_체크_전월      float64
 8   증감율_이용건수_카드론_전월     float64
 9   증감율_이용금액_신용_전월      float64
 10  증감율_이용금액_신판_전월      float64
 11  증감율_이용금액_일시불_전월     float64
 12  증감율_이용금액_할부_전월      float64
 13  증감율_이용금액_CA_전월      float64
 14  증감율_이용금액_체크_전월      float64
 15  증감율_이용금액_카드론_전월     float64
 16  증감율_이용건수_신용_분기      float64
 17  증감율_이용건수_신판_분기      float64
 18  증감율_이용건수_일시불_분기     float64
 19  증감율_이용건수_할부_분기      float64
 20  증감율_이용건수_CA_분기      float64
 21  증감율_이용건수_체크_분기      float64
 22  증감율_이용건수_카드론_분기     float64
 23  증감율_이용금액_신용_분기      float64
 24  증감율_이용금액_신판_분기      float64
 2

### 1.결측치 확인
- 20% 이하
- 결측치 컬럼 삭제 : 혜택수혜율_R3M, 혜택수혜율_B0M

In [13]:
# 결측치 확인
null_check(df) 

null counts : 488,746
null ratio : 20.36%
혜택수혜율_R3M
0.000000     732464
99.999800     66925
1.855113          4
0.864551          4
1.223514          4
              ...  
0.908748          1
1.419543          1
4.781724          1
1.311194          1
1.377071          1
Name: count, Length: 1094468, dtype: int64
################################################################
null counts : 555,522
null ratio : 23.15%
혜택수혜율_B0M
0.000000     799228
99.999800     75729
0.882201          4
2.064493          4
1.796751          4
              ...  
6.750420          1
2.073091          1
3.318849          1
3.937617          1
2.533815          1
Name: count, Length: 957787, dtype: int64
################################################################


In [16]:
drop_cols = ['기준년월', 'ID', '혜택수혜율_R3M', '혜택수혜율_B0M']
df_drop = df.drop(columns = drop_cols)
df_drop.shape

(2400000, 45)

### 인코딩
- 남은 컬럼 모두 수치형이어서 생략

### 2.분산
- 0.01 이상

In [17]:
df_drop.describe()

,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용건수_카드론_전월,증감율_이용금액_신용_전월,증감율_이용금액_신판_전월,증감율_이용금액_일시불_전월,...,잔액_신판ca최대한도소진율_r3m,변동률_일시불평잔,변동률_RV일시불평잔,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M
count,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,...,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06
mean,-2.499633e-02,-2.383989e-02,-1.959183e-02,-8.642645e-02,-1.592283e-02,-2.235531e-02,-2.300829e-03,7.739399e-02,8.808297e-02,9.256195e-02,...,1.874630e-01,8.979931e-01,9.493904e-01,8.766330e-01,9.314377e-01,9.479983e-01,9.385676e-01,-1.006304e-02,3.199591e-03,-3.324818e-03
std,3.206504e-01,3.181794e-01,3.078197e-01,5.119002e-01,1.866209e-01,8.128773e-02,6.779643e-02,4.785608e-01,4.771338e-01,4.619206e-01,...,2.521579e-01,3.611289e-01,2.297757e-01,4.115469e-01,3.048485e-01,2.221469e-01,2.522328e-01,2.427016e-01,2.862019e-01,7.944772e-02
min,-1.999996e+00,-1.999996e+00,-1.999996e+00,-1.999996e+00,-1.999996e+00,-1.199311e+00,-1.999996e+00,-2.004689e+00,-2.004130e+00,-2.003836e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.999996e+00,-1.999996e+00,-1.999996e+00
25%,-7.331375e-02,-7.273252e-02,-6.861702e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.780000e-06,1.900000e-05,2.208000e-05,...,7.132775e-03,8.232759e-01,9.999980e-01,9.005240e-01,9.999980e-01,9.999980e-01,9.999980e-01,-7.272741e-02,-5.386280e-02,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.875955e-01,1.966753e-01,1.941491e-01,...,7.656979e-02,9.712664e-01,9.999980e-01,9.999980e-01,9.999980e-01,9.999980e-01,9.999980e-01,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.876657e-03,9.142750e-03,1.153946e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.552376e-01,2.579933e-01,2.525388e-01,...,2.603923e-01,1.038999e+00,9.999980e-01,9.999980e-01,9.999980e-01,9.999980e-01,9.999980e-01,3.604638e-02,4.484067e-02,0.000000e+00
max,1.999996e+00,1.999996e+00,1.999996e+00,1.999996e+00,1.999996e+00,7.340224e-01,0.000000e+00,2.004175e+00,2.004175e+00,2.003318e+00,...,1.242139e+00,2.062496e+00,1.540538e+00,1.997502e+00,1.996579e+00,1.999996e+00,2.000152e+00,1.999996e+00,1.999996e+00,1.999996e+00


In [21]:
# 분산 확인
variances = df_drop.var()
print(variances.sort_values(ascending=False))
print(variances.describe())
# 2. 분산이 0.01 이하인 컬럼명 리스트
cols_to_drop = variances[variances < 0.01].index.to_list()
X = df_drop.drop(columns=cols_to_drop)


print(f"제거된 컬럼 개수: {len(cols_to_drop)}")
print(f"남은 컬럼 개수: {X.shape[1]}")

증감율_이용건수_할부_분기        0.686693
증감율_이용금액_할부_분기        0.526504
증감율_이용금액_신용_분기        0.389397
증감율_이용금액_신판_분기        0.387919
증감율_이용금액_일시불_분기       0.363612
증감율_이용건수_신용_분기        0.361701
증감율_이용건수_신판_분기        0.357320
증감율_이용건수_일시불_분기       0.342039
증감율_이용건수_할부_전월        0.262042
증감율_이용금액_신용_전월        0.229020
증감율_이용금액_신판_전월        0.227657
증감율_이용금액_일시불_전월       0.213371
증감율_이용금액_할부_전월        0.196633
변동률_할부평잔              0.169371
증감율_이용건수_CA_분기        0.165351
변동률_일시불평잔             0.130414
증감율_이용금액_CA_분기        0.130171
증감율_이용건수_체크_분기        0.128541
증감율_이용금액_체크_분기        0.122795
증감율_이용건수_신용_전월        0.102817
증감율_이용건수_신판_전월        0.101238
증감율_이용건수_일시불_전월       0.094753
증감율_이용금액_카드론_분기       0.093234
변동률_CA평잔              0.092933
증감율_이용건수_카드론_분기       0.091051
변동률_잔액_일시불_B1M        0.081912
잔액_신판ca최대한도소진율_r6m    0.078580
변동률_카드론평잔             0.063621
잔액_신판ca최대한도소진율_r3m    0.063584
잔액_신판최대한도소진율_r6m      0.061012
변동률_잔액_B1M            0.058904
변동률_RV일시불평잔           0.052797
변동률_RVCA

In [22]:
cols_to_drop

['증감율_이용건수_체크_전월', '증감율_이용건수_카드론_전월', '증감율_이용금액_카드론_전월', '변동률_잔액_CA_B1M']

- 표준화

In [23]:
scaler1 = StandardScaler()
scaler1.fit(X)
X_scaler = scaler1.transform(X)
X_scaled_df = pd.DataFrame(X_scaler, columns=X.columns, index=X.index)
X_scaled_df

,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용금액_신용_전월,증감율_이용금액_신판_전월,증감율_이용금액_일시불_전월,증감율_이용금액_할부_전월,증감율_이용금액_CA_전월,...,잔액_신판ca평균한도소진율_r3m,잔액_신판ca최대한도소진율_r3m,변동률_일시불평잔,변동률_RV일시불평잔,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M
0,1.495537,1.503517,1.248519,4.075839,0.085322,0.694524,0.674199,0.477870,4.608017,0.058294,...,3.059667,2.766948,0.149356,0.220248,0.403775,0.223922,0.234078,0.243547,1.120507,0.934839
1,-1.518202,-1.533627,-1.599042,0.168835,0.085322,-2.273926,-2.303129,-2.388678,0.097760,0.058294,...,2.456528,2.519429,1.086719,0.623685,0.070540,0.224900,0.234078,0.243547,-2.279856,-2.353400
2,-0.117635,-0.122183,0.284046,-3.738170,0.085322,-2.046888,-2.075412,0.426487,-4.412497,0.058294,...,2.575045,2.680945,0.086403,0.246910,2.714045,-0.258721,0.234078,0.243547,-0.150196,0.191872
3,0.501181,0.501438,0.563861,-3.738170,0.085322,-1.971424,-1.999723,0.399170,-0.213733,0.058294,...,3.712108,3.545026,0.340519,0.220248,0.422827,0.224503,0.234078,0.243547,0.139614,0.893576
4,0.077955,0.074926,0.063647,0.168835,0.085322,-0.161664,-0.184550,-0.200325,0.097760,0.058294,...,-0.707576,-0.743435,-2.486628,0.220248,0.299759,0.224900,0.234078,0.243547,0.041463,-0.011179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,0.077955,0.074926,0.063647,0.168835,0.085322,-0.161666,-0.184552,-0.200327,0.097760,0.058294,...,-0.612185,-0.614790,0.282461,0.220248,0.299759,0.224900,0.234078,0.243547,0.041463,-0.011179
399996,0.308126,0.306884,0.303412,0.168835,0.085322,0.271032,0.249440,0.247959,0.097760,0.058294,...,-0.356860,-0.336130,-0.783393,0.220248,0.299759,0.224900,0.234078,-0.066743,-0.795990,-0.567232
399997,-0.048138,-0.052147,-0.067702,0.168835,0.085322,0.370069,0.348773,0.296613,-4.412497,0.058294,...,-0.110237,0.011620,0.383180,0.220248,-1.291726,0.224900,0.234078,0.243547,0.154027,0.431099
399998,0.077955,0.074926,0.063647,0.168835,0.085322,-0.161651,-0.184537,-0.200311,0.097760,0.058294,...,-0.658576,-0.693161,0.282461,0.220248,0.299759,0.224900,0.234078,0.243547,0.041463,-0.011179


### 3.다중공선성 제거
- 상관관계 확인
- 0.6 이하

In [30]:
import numpy as np
# 1. 상관행렬 계산
corr_matrix = X_scaled_df.corr().abs()

# 2. 자기 자신 제외 (대각선 제거)
upper = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

# # 3. 상관계수 0.7 이상인 컬럼 찾기
to_drop = [column for column in upper.columns if any(upper[column] > 0.6)]

# 4. 해당 컬럼 제거
X_filtered = X.drop(columns=to_drop)
print(f'남은 컬럼 개수 {len(X_scaled_df.columns)} -> {len(X_filtered.columns)}')

남은 컬럼 개수 41 -> 13


In [31]:
to_drop

['증감율_이용건수_신판_전월',
 '증감율_이용건수_일시불_전월',
 '증감율_이용금액_신용_전월',
 '증감율_이용금액_신판_전월',
 '증감율_이용금액_일시불_전월',
 '증감율_이용금액_할부_전월',
 '증감율_이용건수_신판_분기',
 '증감율_이용건수_일시불_분기',
 '증감율_이용금액_신용_분기',
 '증감율_이용금액_신판_분기',
 '증감율_이용금액_일시불_분기',
 '증감율_이용금액_할부_분기',
 '증감율_이용금액_CA_분기',
 '증감율_이용금액_체크_분기',
 '증감율_이용금액_카드론_분기',
 '잔액_신판최대한도소진율_r6m',
 '잔액_신판평균한도소진율_r3m',
 '잔액_신판최대한도소진율_r3m',
 '잔액_신판ca평균한도소진율_r6m',
 '잔액_신판ca최대한도소진율_r6m',
 '잔액_신판ca평균한도소진율_r3m',
 '잔액_신판ca최대한도소진율_r3m',
 '변동률_일시불평잔',
 '변동률_RV일시불평잔',
 '변동률_CA평잔',
 '변동률_RVCA평잔',
 '변동률_카드론평잔',
 '변동률_잔액_일시불_B1M']

In [32]:
vif_df = vif(X_filtered)

             Feature       VIF
0              const  6.732305
4     증감율_이용금액_CA_전월  1.511395
3     증감율_이용건수_CA_전월  1.500564
1     증감율_이용건수_신용_전월  1.417603
13        변동률_잔액_B1M  1.412393
7     증감율_이용건수_할부_분기  1.195421
12          변동률_할부평잔  1.164089
9     증감율_이용건수_체크_분기  1.135150
5     증감율_이용금액_체크_전월  1.134836
8     증감율_이용건수_CA_분기  1.073382
11  잔액_신판평균한도소진율_r6m  1.070493
2     증감율_이용건수_할부_전월  1.065622
6     증감율_이용건수_신용_분기  1.058432
10   증감율_이용건수_카드론_분기  1.003747


In [33]:
vif_df.to_csv(path+'성과정보_vif.csv',encoding='euc-kr')

- VIF 5 이상인 값 없음

In [36]:
vif_df[vif_df['VIF'] >= 5 ]

,Feature,VIF
0,const,6.732305


In [41]:
X_filtered[vif_df.iloc[1:]['Feature'].tolist()].shape

(2400000, 13)

In [42]:
# 파일 저장
pd.concat([df[['기준년월', 'ID']], X_filtered[vif_df.iloc[1:]['Feature'].tolist()]], axis=1).to_parquet(path + '성과_전처리.parquet')

In [43]:
# 저장된 파일 확인
tmp = pd.read_parquet(path + '성과_전처리.parquet')
tmp.head()

,기준년월,ID,증감율_이용건수_신용_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용금액_CA_전월,증감율_이용금액_체크_전월,증감율_이용건수_신용_분기,증감율_이용건수_할부_분기,증감율_이용건수_CA_분기,증감율_이용건수_체크_분기,증감율_이용건수_카드론_분기,잔액_신판평균한도소진율_r6m,변동률_할부평잔,변동률_잔액_B1M
0,201807,TRAIN_000000,0.454548,1.999996,0.0,0.0,0.0,-0.476453,0.000000,0.0,-1.999996,0.0,0.153562,1.042805,0.261886
1,201807,TRAIN_000001,-0.511808,0.000000,0.0,0.0,0.0,0.226240,0.000000,0.0,0.000000,0.0,0.644283,0.905663,-0.563388
2,201807,TRAIN_000002,-0.062716,-1.999996,0.0,0.0,0.0,0.158594,1.999996,0.0,0.000000,0.0,0.189041,1.993590,-0.046516
3,201807,TRAIN_000003,0.135707,-1.999996,0.0,0.0,0.0,0.120484,0.000000,0.0,0.000000,0.0,0.333548,1.050646,0.023821
4,201807,TRAIN_000004,0.000000,0.000000,0.0,0.0,0.0,-1.999996,0.000000,0.0,-0.404712,0.0,0.021708,0.999998,0.000000


In [45]:
tmp.shape

(2400000, 15)